<a href="https://colab.research.google.com/github/racoope70/daytrading-with-ml/blob/main/AAPL_Day_Trading_ML_Implemented.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install --upgrade --force-reinstall numpy pandas scikit-learn joblib


  Using cached numpy-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached pandas-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached scikit_learn-1.6.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached pytz-2025.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached scipy-1.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
  Using cached six-1.17.0-py2.py3-none-any.whl.metadata (1.7 kB)
Using cached numpy-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
Using cached pandas-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014

In [1]:
# Import Libraries
import numpy as np
import pandas as pd
from google.colab import drive
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


In [3]:
# Mount Google Drive (if using Colab)
drive.mount('/content/drive')

# Load Dataset
file_path = '/content/drive/My Drive/aapl_labeled_trading_dataset.csv'
df = pd.read_csv(file_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Feature Engineering
df['SMA_20'] = df['Close'].rolling(window=20).mean()
df['STD_20'] = df['Close'].rolling(window=20).std()
df['Upper_Band'] = df['SMA_20'] + 2 * df['STD_20']
df['Lower_Band'] = df['SMA_20'] - 2 * df['STD_20']
df['Lowest_Low'] = df['Low'].rolling(window=14).min()
df['Highest_High'] = df['High'].rolling(window=14).max()
df['Stoch'] = ((df['Close'] - df['Lowest_Low']) / (df['Highest_High'] - df['Lowest_Low'])) * 100
df['ROC'] = df['Close'].pct_change(periods=10)
df['OBV'] = (np.sign(df['Close'].diff()) * df['Volume']).cumsum()
typical_price = (df['High'] + df['Low'] + df['Close']) / 3
df['CCI'] = (typical_price - typical_price.rolling(window=20).mean()) / (0.015 * typical_price.rolling(window=20).std())
df['PROC'] = ((df['Close'] - df['Close'].shift(12)) / df['Close'].shift(12)) * 100
df['Rolling_Mean_50'] = df['Close'].rolling(window=50).mean()
df['Expanding_Mean'] = df['Close'].expanding(min_periods=1).mean()


In [5]:
import os

file_path = "/content/drive/My Drive/aapl_labeled_trading_dataset.csv"

# Check if the file exists
if not os.path.exists(file_path):
    raise ValueError(f"Error: File not found at {file_path}")

# Check if the file is empty
if os.stat(file_path).st_size == 0:
    raise ValueError(f"Error: The file {file_path} is empty!")

# Try loading with pandas
df = pd.read_csv(file_path)
print(f"Dataset shape after loading: {df.shape}")

# Show first few rows
print(df.head())


Dataset shape after loading: (4873, 32)
                    Datetime        Open        High         Low       Close  \
0  2022-04-13 13:30:00+00:00  167.520004  168.899994  166.770004  168.690002   
1  2022-04-13 14:30:00+00:00  168.699997  170.000000  168.285004  169.690002   
2  2022-04-13 15:30:00+00:00  169.699997  171.000000  169.600098  170.440094   
3  2022-04-13 16:30:00+00:00  170.449997  171.000000  170.279999  170.559998   
4  2022-04-13 17:30:00+00:00  170.570007  170.593597  169.949997  170.279999   

     Volume      EMA_10      EMA_50      SMA_20      SMA_50  ...  Sell_Signal  \
0  15711444  167.876788  170.854719  168.163296  171.580093  ...            1   
1   9751613  168.206463  170.809044  168.065046  171.426693  ...            1   
2  10069224  168.612578  170.794576  168.046050  171.282595  ...            1   
3   6851724  168.966654  170.785376  168.076545  171.140594  ...            1   
4   6078766  169.205444  170.765558  168.053545  171.001602  ...          